<a href="https://colab.research.google.com/github/gcasaldi/CS50/blob/main/Un_benvenuto_a_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
!pip install openpyxl
import pandas as pd
from datetime import datetime, timedelta
from google.colab import files
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.utils import to_categorical
from collections import Counter





In [23]:
from google.colab import files

# Carica il file Excel
uploaded = files.upload()
file_name = 'it-lotto-past-draws-archive.xls'

Saving it-lotto-past-draws-archive.xls to it-lotto-past-draws-archive.xls


In [22]:
# Carica il file Excel
uploaded = files.upload()
file_name = 'it-lotto-past-draws-archive.xls'
file_path = '/content/it-lotto-past-draws-archive.xls'

def carica_database(file_path):
    df = pd.read_excel(file_path, engine='xlrd', skiprows=1)
    numeri_estratti = df.select_dtypes(include='number').dropna(axis=1, how='all')
    numeri_estratti = numeri_estratti.tail(500)
    return numeri_estratti

def calcola_frequenze(df):
    frequenze = df.stack().value_counts()
    return frequenze.head(5).index.tolist()

def calcola_ritardatari(df):
    numeri_totali = set(range(1, 91))
    ultime_uscite = {}
    for numero in numeri_totali:
        try:
            ultima_pos = df.apply(lambda colonna: numero in colonna.values).iloc[::-1].idxmax()
            ultime_uscite[numero] = ultima_pos
        except:
            ultime_uscite[numero] = -1
    ritardatari = sorted(ultime_uscite, key=ultime_uscite.get, reverse=True)[:5]
    return ritardatari



Saving it-lotto-past-draws-archive.xls to it-lotto-past-draws-archive (1).xls


In [23]:
# Carica i dati
df = carica_database(file_path)

# Preprocessing
scaler = MinMaxScaler(feature_range=(0, 1))
numeri_scalati = scaler.fit_transform(df.values)

*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'


In [24]:
import pandas as pd
import zipfile
# Assicurati di avere openpyxl installato
!pip install openpyxl

import pandas as pd

# Assicurati di avere openpyxl installato
!pip install openpyxl

# Leggi il file Excel in un DataFrame specificando l'engine
try:
    df = pd.read_excel('/content/it-lotto-past-draws-archive.xls', engine='xlrd')
except ValueError as e:
    print(f"Errore durante la lettura del file: {e}")
    exit()

# Controlla i nomi delle colonne
print("Nomi delle colonne nel DataFrame:", df.columns)

# Seleziona le colonne rilevanti (sostituisci con i nomi delle colonne effettive)
# Assicurati che le colonne selezionate siano quelle con i numeri estratti
try:
    numeri_estratti = df[['Colonna1', 'Colonna2', 'Colonna3', 'Colonna4', 'Colonna5']]
except KeyError as e:
    print(f"Errore: una o più colonne non trovate: {e}")
    exit()
    exit()

# Controlla i nomi delle colonne
print("Nomi delle colonne nel DataFrame:", df.columns)

# Seleziona le colonne rilevanti (sostituisci con i nomi delle colonne effettive)
# Assicurati che le colonne selezionate siano quelle con i numeri estratti
try:
    numeri_estratti = df[['Colonna1', 'Colonna2', 'Colonna3', 'Colonna4', 'Colonna5']]
except KeyError as e:
    print(f"Errore: una o più colonne non trovate: {e}")
    exit()

ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 447, in run
    conflicts = self._determine_conflicts(to_install)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 578, in _determine_conflicts
    return check_install_conflicts(to_install)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/operations/check.py", line 101, in check_install_conflicts
    package_set, _ = create_package_

In [11]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.utils import to_categorical
from collections import Counter  # Per contare la frequenza dei numeri

# Carica i dati
df = pd.read_excel('/content/it-lotto-past-draws-archive.xls', engine='xlrd', skiprows=1)
numeri_estratti = df.select_dtypes(include='number').dropna(axis=1, how='all').tail(500)  # Ultimi 500 colpi

# Funzione per calcolare la frequenza dei numeri
def calcola_frequenze(df):
    numeri_frequenti = df.stack().value_counts()
    return numeri_frequenti.head(5).index.tolist()

# Funzione per calcolare i numeri ritardatari
def calcola_ritardatari(df):
    numeri_ritardatari = df.apply(lambda col: len(col) - col[::-1].first_valid_index(), axis=0)
    return numeri_ritardatari.sort_values().head(5).index.tolist()

# Preprocessing
scaler = MinMaxScaler(feature_range=(0, 1))
numeri_scalati = scaler.fit_transform(numeri_estratti.values)

# Creazione delle sequenze temporali
X = []
y = []
sequence_length = 10

for i in range(sequence_length, len(numeri_scalati)):
    X.append(numeri_scalati[i-sequence_length:i])
    y.append(numeri_scalati[i])

X = np.array(X) # Now np.array should work correctly
y = np.array(y)

# Modello LSTM
model = Sequential()
model.add(LSTM(64, activation='relu', input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(X.shape[2], activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Allena il modello
y_onehot = to_categorical(y.argmax(axis=1), num_classes=X.shape[2])
model.fit(X, y_onehot, epochs=100, batch_size=32, verbose=1)

# Previsione LSTM
pred = model.predict(X[-1].reshape(1, sequence_length, X.shape[2]))
numeri_previsti_lstm = np.argsort(pred[0])[-5:] + 1

# Calcola frequenze e ritardatari
numeri_frequenti = calcola_frequenze(df)
numeri_ritardatari = calcola_ritardatari(df)

# Combina le previsioni
all_predictions = np.concatenate([numeri_previsti_lstm, numeri_frequenti, numeri_ritardatari])
predicted_numbers_freq = Counter(all_predictions)
previsione_finale = [num for num, freq in predicted_numbers_freq.most_common(5)]

print("🔥 Numeri Predetti da LSTM:", numeri_previsti_lstm)
print("Numeri più frequenti:", numeri_frequenti)
print("Numeri ritardatari:", numeri_ritardatari)
print("Previsione Finale:", previsione_finale)



*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'
Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.0000e+00 - loss: nan
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.0000e+00 - loss: nan
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.0000e+00 - loss: nan
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.0000e+00 - loss: nan
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.0000e+00 - loss: nan
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.0000e+00 - loss: nan
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.0000e+00 - loss: nan
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.0000e+00 - loss: nan
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.0000e+00 - loss: nan
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.0000e+00 - loss: nan
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.0000e+00 - loss: nan
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.0000e+00 - l

In [12]:
# prompt: dopo aver fatto il training calcola nuovamente la predizione in modo che sia sicuro dell'estrazione dei numeri

import numpy as np

# ... (preceding code)

# Previsione (modificata per maggiore sicurezza)
last_sequence = X[-1].reshape(1, sequence_length, X.shape[2])
predictions = []
for _ in range(5):  # Genera 5 previsioni per avere più sicurezza
    pred = model.predict(last_sequence)
    predicted_numbers = np.argsort(pred[0])[-5:] + 1
    predictions.append(predicted_numbers)
    # Aggiorna la sequenza per la prossima previsione (usando l'ultima previsione)
    next_step = pred.reshape(1, X.shape[2])
    last_sequence = np.concatenate((last_sequence[:, 1:, :], next_step.reshape(1,1,X.shape[2])), axis = 1)


# Calcola i numeri più frequenti nelle previsioni
from collections import Counter
all_predictions = np.concatenate(predictions)
predicted_numbers_freq = Counter(all_predictions)

# Seleziona i 5 numeri più frequenti
top_5_predicted = predicted_numbers_freq.most_common(5)
final_prediction = [num for num, freq in top_5_predicted]

print("Numeri Predetti (con maggiore sicurezza):", final_prediction)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Numeri Predetti (con maggiore sicurezza): [18, 19, 20, 43, 44]


In [14]:

import pandas as pd
import datetime

file_path = '/content/it-lotto-past-draws-archive.xls'

def carica_database(file_path):
    # Leggi il file Excel
    df = pd.read_excel(file_path, engine='xlrd', skiprows=1)
    # Prende solo numeri e rimuove colonne vuote
    numeri_estratti = df.select_dtypes(include='number').dropna(axis=1, how='all')

    # Usa solo le ultime 50 estrazioni 🔥
    numeri_estratti = numeri_estratti.tail(50)
    return numeri_estratti

def calcola_frequenze(df):
    frequenze = df.stack().value_counts()
    return frequenze.head(5).index.tolist()

def calcola_ritardatari(df):
    numeri_totali = set(range(1, 91))
    ultime_uscite = {}

    for numero in numeri_totali:
        try:
            ultima_pos = df.apply(lambda colonna: numero in colonna.values).iloc[::-1].idxmax()
            ultime_uscite[numero] = ultima_pos
        except:
            ultime_uscite[numero] = -1

    ritardatari = sorted(ultime_uscite, key=ultime_uscite.get, reverse=True)[:5]
    return ritardatari

def somma_90(previsione):
    somma = (previsione[0] + previsione[1]) % 90
    if somma == 0:
        somma = 90
    return somma

# Esegui tutto
df = carica_database(file_path)
numeri_frequenti = calcola_frequenze(df)
numeri_ritardatari = calcola_ritardatari(df)
previsione = numeri_frequenti + numeri_ritardatari
somma = somma_90(previsione)

print(f"Data aggiornamento: {datetime.date.today()}")
print("Numeri più frequenti:", numeri_frequenti)
print("Numeri ritardatari:", numeri_ritardatari)
print("Previsione Finale:", previsione)
print("🔥 Somma 90:", somma)



*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'
Data aggiornamento: 2025-03-06
Numeri più frequenti: [80.0, 7.0, 87.0, 23.0, 2.0]
Numeri ritardatari: [1, 4, 7, 8, 13]
Previsione Finale: [80.0, 7.0, 87.0, 23.0, 2.0, 1, 4, 7, 8, 13]
🔥 Somma 90: 87.0


In [8]:
# Genera una previsione (esempio semplificato)
# La previsione si basa sui numeri più frequenti
previsione = numeri_frequenti

In [4]:
# Crea un DataFrame per la previsione
previsione_df = pd.DataFrame({'Numero': previsione})


In [9]:
# Salva la previsione in un file CSV
previsione_df.to_csv('/content/previsione_lotto.csv', index=False)

print("Previsione salvata in /content/previsione_lotto.csv")

from google.colab import files
files.download('/content/previsione_lotto.csv')


Previsione salvata in /content/previsione_lotto.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>